In [5]:
%load_ext autoreload
%autoreload 2

In [8]:
pip install --upgrade tqdm

  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.59.0
    Uninstalling tqdm-4.59.0:
      Successfully uninstalled tqdm-4.59.0
Note: you may need to restart the kernel to use updated packages.


In [70]:
import numpy as np
from pyldpc import make_ldpc, encode, decode, get_message

n = 20
d_v = 4
d_c = 5
snr = 200
H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)
k = G.shape[1]
v = np.random.randint(2, size=k)
y = encode(G, v, snr)
d = decode(H, y, snr)
x = get_message(G, d)
assert abs(x - v).sum() == 0

In [ ]:
i=0
while v.shape != (8,):
    i+=1
    H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)
    v = np.random.randint(2, size=G.shape[1])
    print('aaaa')
print(v.shape)
print(i)

In [78]:
from decoder import bit_flip_dec

In [4]:
decoded_x = bit_flip_dec(H,y)

100%|███████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 21708.75it/s]


In [6]:
abs(x - decoded_x).sum() #comparison with pyldpc decoder

ValueError: operands could not be broadcast together with shapes (6,) (15,) 

In [8]:
abs(x - v).sum() #comparison with input msg

(15,)

In [90]:
y.shape

(20,)

In [ ]:
from abc import ABC
class BaseAlgo(ABC):
    def __init__(self, algo_name , algo_params = {}):
        super(BaseAlgo, self).__init__()
        self.algo_name = algo_name
        self.algo_params = algo_params
        
    
    def decode(self, y_err):
        pass

In [91]:
class Stats():
    def __init__(self, l1_dist, absolute_correct, is_codeword):
        self.l1_dist = l1_dist
        self.absolute_correct = absolute_correct
        self.is_codeword = is_codeword

In [93]:
import itertools

def benchmark(algorithm, codes_per_case, trials_per_code):
    stats = {}
    n = [10, 100, 1000]
    # rate = [0.01, 0.1, 0.5]
    frac_of_errs = [0.1, 0.25, 0.4]
    cases = list(itertools.product(n, frac_of_errs))
    d_v = 4
    d_c = 5
    snr = 200


    for case in cases:
        n, frac_of_errs = case
        
        for code in range(codes_per_case):
            H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)
            k = G.shape[1]
            v = np.random.randint(2, size=k)
            y = encode(G, v, snr)
            
            l1_dist = 0
            num_abs_correct = 0
            num_valid_codeword = 0
            
            for trial in range(trials_per_code):
                y = add_error(v, int(frac_of_errs*n))
                decode_y = algorithm(H,y)
                l1_dist += np.absolute(decode_y - y)
                num_abs_correct += 1 if decode_y == y else 0
                num_valid_codeword += 1 if (H@decode_y).sum() == 0 else 0
            
            stats[code] = Stats(l1_dist/trials_per_code, num_abs_correct/trials_per_code, num_valid_codeword/trials_per_code)

In [12]:
benchmark(1,1,1)

[(10, 0.01, 0.1), (10, 0.01, 0.25), (10, 0.01, 0.4), (10, 0.1, 0.1), (10, 0.1, 0.25), (10, 0.1, 0.4), (10, 0.5, 0.1), (10, 0.5, 0.25), (10, 0.5, 0.4), (100, 0.01, 0.1), (100, 0.01, 0.25), (100, 0.01, 0.4), (100, 0.1, 0.1), (100, 0.1, 0.25), (100, 0.1, 0.4), (100, 0.5, 0.1), (100, 0.5, 0.25), (100, 0.5, 0.4), (1000, 0.01, 0.1), (1000, 0.01, 0.25), (1000, 0.01, 0.4), (1000, 0.1, 0.1), (1000, 0.1, 0.25), (1000, 0.1, 0.4), (1000, 0.5, 0.1), (1000, 0.5, 0.25), (1000, 0.5, 0.4)]
